# Processo de Machine Learning

### Pré-processamento e Separação dos Dados

#### Variáveis
- Escolha das **variáveis com entradas categóricas**:
    - Não foram adicionadas ao modelo as variáveis 'reservation_status', 'reservation_status_date' e 'matching_room_type' devido a Data Leakage. O modelo receberia uma informação importante no que tange à situação do cancelamento. As variáveis 'company'e 'agent' não possuem informações relevantes por motivos de sigilo.


- Escolha das **variáveis com entradas numéricas**: 
    - Foi escolhido arrival_date_week_number pois essa variável representa a mesma informação que as outras três variáveis de data de chegada que também não foram selecionadas ('arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month').
    
    
- Devido ao período do conjunto de dados, os meses de julho e agosto são os únicos que possuem entradas a mais que os outros meses (conforme comentado anteriormente na EDA). Por isso, essas entradas foram removidas.
    

#### Valores Missing e Repetidos.

- A quantidade de valores nulos é muito pequena. Portanto, foram removidos.


- Devido às características do negócio, é comum que haja entradas duplicadas. Poranto, não foram removidas. Ainda assim, se as removesse, ocasionaria em uma perda muito grande de informação (quase 30% dos dados).


- As entradas de dadas que não possuem nenhum valor para adultos, crianças ou bebês foram removidas, pois se configuraria como um erro no preenchimento dos dados, afinal não faz sentido reservas compostas por ninguém. 

    
    
#### Métricas

- 0 - Não-cancelado (63%)  |   1 - Cancelado (27%). Desbalanceamento de classe.


- Se considerássemos um aumento no número de falsos positivos (redução da precisão), ou seja, o modelo iria prever cancelamento de reservas sendo que as mesmas na verdade não foram canceladas, as possíveis consequências seriam:
    - Falta de planejamnto na estrutura do hotel para receber os hóspedes que na realidade não foram cancelados. Ocorrência de Overbooking.      
          
          
- Se considerássemos um aumento no número de falsos negativos (redução do recall), ou seja, o modelo iria prever que não houve cancelamentos apesar desses cancelamentos terem de fato ocorrido, as possíveis consequências seriam: 
    - O planejamento do negócio seria afetado com a expectativa de renda futura que não se concretizaria. Nesse contexto, a renda esperada não existiria e possíveis rendas poderiam ser perdidas devido a não agendamentos realizados por consequência da pseudo-ocupação dos quartos.  
      
      
- Percebe-se que em ambas as situações as consequências podem ser muito ruins para o negócio. Se o gestor prioriza o lucro, ou necessita urgentemente auferir receitas por qualquer que seja o motivo específico, a priorização da métrica Recall (Sensibilidade) seria o mais recomendado. Contudo, se há uma preocupação com a imagem do hotel para com os clientes, focando na fidelização dos mesmos através de serviços bem prestados com o mínimo de intercorrências no processo das reservas, a priorização da métrica Precision (Precisão) é o mais recomendado.  


- Por ser uma decisão bastante específica, onde não há prepoderância significativa de argumentos, foi escolhido a priorização da métria ***F1-score***, mas sempre **considerando** um melhor **equilíbrio** entre **precision/recall**. Uma média harmônica entre Precision e Recall. O que contribuiu também para essa escolha é o fato das classes serem desbalanceadas.

### Construção dos Modelos 


#### Encoders
- Foi testada, nos modelos base Logistic Regression e Decision Tree, a utilização do **RareLabelEncoder** para **diminuição** da **cardinalidade** da feature **'country'**. O impacto não foi muito significativo nas métricas, apesar de reduzir a cardinalidade de 177 para 23 sem perder muita informação (manteve 95% dos dados "originais" intactos). Contudo, temos um modelo com menos features, **mais interpretável** e de certa forma mais rápido. Portanto, **optou-se por utilizar esse encoder**. Além disso, a grande maioria das categorias dessa variável possuíam uma frequência muito pequena. Mais um fato que motivou essa tomada de decisão.



- Foi testado nos modelos base Logistic Regression e Decision Tree combinações diferentes dos encoders OneHotEncoder e JamesSteinEncoder como exploração inicial de modelos. As combinações cogitadas foram:
    1. OneHotEncoder para todas as variáveis categóricas.
    2. JamesSteinEncoder para a variável 'country' e OneHotEncoder para o restante das variáveis categóricas.
    Buscou-se ter o melhor modelo nessas circunstâncias para se fazer uma comparação.
    Não foi selecionado OneHotEncoder para o modelo base GaussianNB pois geraria uma matriz esparsa, e esse modelo não consegue performar com esse tipo de matriz.



- **Apenas** a utilização do **OneHotEncoder** foi preferida em vez de JamesSteinEncoder para a variável 'country' e OneHotEncoder para as demais (exceto GaussianNB). Essa tentativa de mais diminuição de cardinalidade na variável 'country' com JamesSteinEncoder não gerou impacto signifiativo. Contudo, pela natureza desse encoder, informações da variável target são vazadas para as features, o tornando de algum modo menos consistente e verossímil, o que não é ideal. Por isso OneHotEncoder foi **escolhido**. 



- Os Encoders que foram decididos nos modelos base (Logistic Regression, Decision Tree, Gaussian Naive Bayes) foram estabelecidos também nos respectivos modelos Ensemble, por questões de padronização para comparação.


#### Escolha dos Hiperparâmetros e Testagem/Validação
- **Todos** os modelos foram criados com **validação cruzada**. O **preprocessamento** dos dados foram executados **dentro de cada validação** para trazer maior aproximação com a realidade e para **evitar Data Leakage**. Para a escolha dos hiperparâmetros foi usada uma procura randomizada dos mesmos (RandomizedSearchCV) para processar de maneira mais rápida.


- Foi testado pesos para cada classe, afinal, há um desbalanceamento. Os valores dos pesos escolhidos para serem testados foram ligeiramente diferentes dos valores de pesos em equilíbrio de classe (*balanced*). Estes pesos em equilíbrio são o inverso da frequência das respectivas classes (Peso de 0.37 para classe 0 e de 0.63 para classe 1). Importante frisar que o range de *class_weight* selecionado escalona priorizando a classe minoritária. Não é recomendável se distanciar muito dos pesos em equilíbrio pois o modelo pode passar a ter dificuldade de lidar com a classe que recebeu o menor peso, tornando-se ineficiente e novamente desbalanceado, enviesado.



- Encontrados os hiperparâmetros ideais, foi testado o modelo com os mesmos. Buscou-se testar sua **capacidade de generalização** com dados não vistos durante todo o processo, para basicamente simular o **modelo em produção**.



- Os modelos que fazem parte do VotingClassifier foram configurados com os hiperparâmetros ideais entrontrados. Todos os modelos ensemble que o compõe possuem cem estimadores.



- A configuração da Decision Tree que compõe o GradientBoostingClassifier é com base nos hiperparâmetros ideais encontrados.

### Avaliação dos Modelos
- De modo geral, **todos os modelos generalizaram de maneira consistente**, aproximando-se de suas respectivas métricas de teste. Importante frisar que generalizar consistentemente se refere à fidelidade às suas métricas de teste (oriundas da validação cruzada) quando testados com os dados de teste, não efetivamente à capacidade/qualidade de generalização.


- Todos os ensembles possuíram 100 estimadores.



- Ao analisar as feature_importances da Random Forest, por questão de facilitação para adquirir do nome das features, foi escalonado os dados da mesma. De fato, a normalização/padronização nesses tipos de modelo são desnecessárias e não geram impacto ao faze-la. Por conta disso, por questão de padronização para comparação, todos os modelos do projeto baseados em modelos de árvore foram também padronizados/normalizados.


- Com relação à importância das features no Random Forest:
    - A maioria das variáveis que não influenciam o modelo são aquelas oriundas do encoding de variáveis com categorias com baixíssimas frequências. Especialmente da coluna 'country' (variável com maior cardinalidade). 

    - A variável "**days_in_waiting_list**" não influencia o modelo, apesar de não possuir alta cardinalidade (é uma variável numérica). Por si só não se tornou relevante nesse contexto. Importante salientar que na análise feita na exploração dos dados, aparentemente se notou uma possível relação entre o número de dias na lista de espera e a possibilidade de cancelamento. Contudo, também foi amostrado que praticamente 97% das entradas nessa variável foram de mesmo valor, demonstrando uma variância baixíssima e justificando a ineficiência dentro do modelo preditivo. Outras variáveis que por si só  também não foram relevantes: **'babies'** e **'is_repeated_guest'**.

    - Importante ter em mente que a análise por feature_importances com OneHotEncoder fica muito atrelada à frequência da variável que sofreu o encoding. Dentre as variáveis que não sofreram encoding as que **mais impactam** no cancelamento ou não da reserva são o número de dias decorridos entre a data de entrada da reserva no PMS (Property Management System) e a data de chegada no hotel (**lead_time**), a taxa diária média (**adr**) e o número da semana de chegada no ano (**arrival_date_week_number**). Essas duas primeiras são também as que mais influenciam todo o modelo e essa última a quarta que mais influencia.


- Com relação aos coeficientes da Regressão Logística: As **variáveis** com barras **verdes** (**coeficientes positivos**) são aquelas que influenciam positivamente o modelo, ou seja, **quanto maior, maior a chance de cancelar a reversa**. As variáveis **vermelhas** (**coeficientes negativos**) influenciam negativamente, ou seja, **quanto maior, maior a chance de não cancelar**.



- O **pior** modelo foi **Gaussian Naive Bayes**. Os **melhores** modelos foram os ensembles de modelos de árvore. Importante notar que o XGBoost, um dos modelos mais poderosos existentes para dados tabulares, desempenhou um pouco pior do que o Random Forest.




- No conjunto dos **modelos que melhor performaram**, todos tiveram **métricas muito próximas**. Para garantir uma maior **segurança e estabilidade**, criou-se um ensemble **VotingClassifier** composto pelos modelos ensemble. 



- As **curvas** do **Random Forest** e do **Voting Classifier** (em ambos os gráficos) praticamente se **sobrepusseram**, demonstrando um desempenho praticamente idêntico. 



- O **VotingClassifier** foi considerado o **modelo ideal**.


